#Implementing SCAD

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
import seaborn as sns
import numpy as np
import pandas as pd
from sklearn.linear_model import Ridge, Lasso, ElasticNet, LinearRegression
from sklearn.datasets import make_spd_matrix
from scipy.optimize import minimize
from scipy.linalg import toeplitz
from sklearn.metrics import mean_absolute_error as mae, mean_squared_error as mse, r2_score as R2

device = torch.device("cpu")
dtype = torch.float64

In [2]:
# beta hat is coeff
# lambda is strength of regularization var
# alpha is number of reduced coeffs

def scad_penalty(beta_hat, lambda_val, a_val):
    is_linear = (np.abs(beta_hat) <= lambda_val)
    is_quadratic = np.logical_and(lambda_val < np.abs(beta_hat), np.abs(beta_hat) <= a_val * lambda_val)
    is_constant = (a_val * lambda_val) < np.abs(beta_hat)

    linear_part = lambda_val * np.abs(beta_hat) * is_linear
    quadratic_part = (2 * a_val * lambda_val * np.abs(beta_hat) - beta_hat**2 - lambda_val**2) / (2 * (a_val - 1)) * is_quadratic
    constant_part = (lambda_val**2 * (a + 1)) / 2 * is_constant
    return linear_part + quadratic_part + constant_part

def scad_derivative(beta_hat, lambda_val, a_val):
    return lambda_val * ((beta_hat <= lambda_val) + (a_val * lambda_val - beta_hat)*((a_val * lambda_val - beta_hat) > 0) / ((a_val - 1) * lambda_val) * (beta_hat > lambda_val))

In [3]:
class SCAD(nn.Module):

    def __init__(self, input_size, alpha=1.0, l1_ratio=0.5):
        """
        Initialize the ElasticNet regression model.

        Args:
            input_size (int): Number of input features.
            alpha (float): Regularization strength. Higher values of alpha
                emphasize L1 regularization, while lower values emphasize L2 regularization.
            l1_ratio (float): The ratio of L1 regularization to the total
                regularization (L1 + L2). It should be between 0 and 1.

        """
        super(SCAD, self).__init__()
        self.input_size = input_size
        self.alpha = alpha
        self.l1_ratio = l1_ratio

        # Define the linear regression layer
        self.linear = nn.Linear(input_size, 1,bias=False,device=device,dtype=dtype)

    def scad_penalty(self, beta_hat, lambda_val, a_val):
      is_linear = (torch.abs(beta_hat) <= lambda_val)
      is_quadratic = np.logical_and(lambda_val < torch.abs(beta_hat), torch.abs(beta_hat) <= a_val * lambda_val)
      is_constant = (a_val * lambda_val) < torch.abs(beta_hat)

      linear_part = lambda_val * torch.abs(beta_hat) * is_linear
      quadratic_part = (2 * a_val * lambda_val * torch.abs(beta_hat) - beta_hat**2 - lambda_val**2) / (2 * (a_val - 1)) * is_quadratic
      constant_part = (lambda_val**2 * (a_val + 1)) / 2 * is_constant
      return linear_part + quadratic_part + constant_part

    def scad_derivative(self, beta_hat, lambda_val, a_val):
        return lambda_val * ((beta_hat <= lambda_val) + (a_val * lambda_val - beta_hat)*((a_val * lambda_val - beta_hat) > 0) / ((a_val - 1) * lambda_val) * (beta_hat > lambda_val))

    def forward(self, x):
        """
        Forward pass of the ElasticNet model.

        Args:
            x (Tensor): Input data with shape (batch_size, input_size).

        Returns:
            Tensor: Predicted values with shape (batch_size, 1).

        """
        return self.linear(x)

    def loss(self, y_pred, y_true):
        """
        Compute the ElasticNet loss function.

        Args:
            y_pred (Tensor): Predicted values with shape (batch_size, 1).
            y_true (Tensor): True target values with shape (batch_size, 1).

        Returns:
            Tensor: The ElasticNet loss.

        """
        #l1_reg = torch.norm(self.linear.weight, p=1,dtype=torch.float64)
        loss = self.scad_penalty(y_pred, .5, .5)
        return loss


    def fit(self, X, y, num_epochs=100, learning_rate=0.01):
        """
        Fit the ElasticNet model to the training data.

        Args:
            X (Tensor): Input data with shape (num_samples, input_size).
            y (Tensor): Target values with shape (num_samples, 1).
            num_epochs (int): Number of training epochs.
            learning_rate (float): Learning rate for optimization.

        """
        optimizer = optim.SGD(self.parameters(), lr=learning_rate)

        for epoch in range(num_epochs):
            self.train()
            optimizer.zero_grad()
            y_pred = self(X)
            loss = self.loss(y_pred, y)
            loss.sum().backward()
            optimizer.step()

   #         if (epoch + 1) % 100 == 0:
          #      print(f"Epoch [{epoch + 1}/{num_epochs}], Loss: {loss.item()}")

    def predict(self, X):
        """
        Predict target values for input data.

        Args:
            X (Tensor): Input data with shape (num_samples, input_size).

        Returns:
            Tensor: Predicted values with shape (num_samples, 1).

        """
        self.eval()
        with torch.no_grad():
            y_pred = self(X)
        return y_pred
    def get_coefficients(self):
        """
        Get the coefficients (weights) of the linear regression layer.

        Returns:
            Tensor: Coefficients with shape (output_size, input_size).

        """
        return self.linear.weight


# Real World Data Application

In [14]:
data = pd.read_csv('cars(2).csv')

In [16]:
data

,MPG,CYL,ENG,WGT
0,18.0,8,307.0,3504
1,15.0,8,350.0,3693
2,18.0,8,318.0,3436
3,16.0,8,304.0,3433
4,17.0,8,302.0,3449
...,...,...,...,...
387,27.0,4,140.0,2790
388,44.0,4,97.0,2130
389,32.0,4,135.0,2295
390,28.0,4,120.0,2625


In [63]:
x = data.drop(columns=['MPG']).values
y = data['MPG'].values
len(y)

392

In [64]:
# put your data in torch tensors
x = torch.tensor(x,device=device)
y = torch.tensor(y,device=device)
len(x)

392

In [65]:
model = SCAD(input_size=x.shape[1],alpha=0.01)

In [66]:
model.fit(x,y,num_epochs=2000,learning_rate=0.01)

In [67]:
model.get_coefficients()[0].cpu().detach().numpy()

array([-0.17847327,  0.29116787,  0.39727948])

# Implement ElasticNet and LASSO as well:


In [97]:
class ElasticNet(nn.Module):
    def __init__(self, input_size, alpha=1.0, l1_ratio=0.5):
        """
        Initialize the ElasticNet regression model.

        Args:
            input_size (int): Number of input features.
            alpha (float): Regularization strength. Higher values of alpha
                emphasize L1 regularization, while lower values emphasize L2 regularization.
            l1_ratio (float): The ratio of L1 regularization to the total
                regularization (L1 + L2). It should be between 0 and 1.

        """
        super(ElasticNet, self).__init__()
        self.input_size = input_size
        self.alpha = alpha
        self.l1_ratio = l1_ratio

        # Define the linear regression layer
        self.linear = nn.Linear(input_size, 1,bias=False,device=device,dtype=dtype)

    def forward(self, x):
        """
        Forward pass of the ElasticNet model.

        Args:
            x (Tensor): Input data with shape (batch_size, input_size).

        Returns:
            Tensor: Predicted values with shape (batch_size, 1).

        """
        return self.linear(x)

    def loss(self, y_pred, y_true):
        """
        Compute the ElasticNet loss function.

        Args:
            y_pred (Tensor): Predicted values with shape (batch_size, 1).
            y_true (Tensor): True target values with shape (batch_size, 1).

        Returns:
            Tensor: The ElasticNet loss.

        """
        mse_loss = nn.MSELoss()(y_pred, y_true)
        l1_reg = torch.norm(self.linear.weight, p=1)
        l2_reg = torch.norm(self.linear.weight, p=2)

        objective = (1/2) * mse_loss + self.alpha * (
            self.l1_ratio * l1_reg + (1 - self.l1_ratio) * (1/2)*l2_reg**2)

        return objective

    def fit(self, X, y, num_epochs=100, learning_rate=0.01):
        """
        Fit the ElasticNet model to the training data.

        Args:
            X (Tensor): Input data with shape (num_samples, input_size).
            y (Tensor): Target values with shape (num_samples, 1).
            num_epochs (int): Number of training epochs.
            learning_rate (float): Learning rate for optimization.

        """
        optimizer = optim.SGD(self.parameters(), lr=learning_rate)

        for epoch in range(num_epochs):
            self.train()
            optimizer.zero_grad()
            y_pred = self(X)
            loss = self.loss(y_pred, y)
            loss.backward()
            optimizer.step()

            if (epoch + 1) % 100 == 0:
                print(f"Epoch [{epoch + 1}/{num_epochs}], Loss: {loss.item()}")

    def predict(self, X):
        """
        Predict target values for input data.

        Args:
            X (Tensor): Input data with shape (num_samples, input_size).

        Returns:
            Tensor: Predicted values with shape (num_samples, 1).

        """
        self.eval()
        with torch.no_grad():
            y_pred = self(X)
        return y_pred
    def get_coefficients(self):
        """
        Get the coefficients (weights) of the linear regression layer.

        Returns:
            Tensor: Coefficients with shape (output_size, input_size).

        """
        return self.linear.weight

In [112]:
class SqrtLasso(nn.Module):
    def __init__(self, input_size, alpha=0.1):
        """
        Initialize the  regression model.


        """
        super(SqrtLasso, self).__init__()
        self.input_size = input_size
        self.alpha = alpha


        # Define the linear regression layer
        self.linear = nn.Linear(input_size, 1,bias=False,device=device,dtype=dtype)

    def forward(self, x):
        """
        Forward pass of the model.

        Args:
            x (Tensor): Input data with shape (batch_size, input_size).

        Returns:
            Tensor: Predicted values with shape (batch_size, 1).

        """
        return self.linear(x)

    def loss(self, y_pred, y_true):
        """
        Compute the loss function.

        Args:
            y_pred (Tensor): Predicted values with shape (batch_size, 1).
            y_true (Tensor): True target values with shape (batch_size, 1).

        Returns:
            Tensor: The loss.

        """
        mse_loss = nn.MSELoss()(y_pred, y_true)
        l1_reg = torch.norm(self.linear.weight, p=1,dtype=torch.float64)
        # l2_reg = torch.norm(self.linear.weight, p=2,dtype=torch.float64)

        loss = torch.sqrt(mse_loss) + self.alpha * (l1_reg)

        return loss

    def fit(self, X, y, num_epochs=200, learning_rate=0.01):
        """
        Fit the model to the training data.

        Args:
            X (Tensor): Input data with shape (num_samples, input_size).
            y (Tensor): Target values with shape (num_samples, 1).
            num_epochs (int): Number of training epochs.
            learning_rate (float): Learning rate for optimization.

        """
        optimizer = optim.Adam(self.parameters(), lr=learning_rate)

        for epoch in range(num_epochs):
            self.train()
            optimizer.zero_grad()
            y_pred = self(X)
            loss = self.loss(y_pred, y)
            loss.backward()
            optimizer.step()

            if (epoch + 1) % 100 == 0:
                print(f"Epoch [{epoch + 1}/{num_epochs}], Loss: {loss.item()}")

    def predict(self, X):
        """
        Predict target values for input data.

        Args:
            X (Tensor): Input data with shape (num_samples, input_size).

        Returns:
            Tensor: Predicted values with shape (num_samples, 1).

        """
        self.eval()
        with torch.no_grad():
            y_pred = self(X)
        return y_pred
    def get_coefficients(self):
        """
        Get the coefficients (weights) of the linear regression layer.

        Returns:
            Tensor: Coefficients with shape (output_size, input_size).

        """
        return self.linear.weight

# Generated Dataset Comparison

In [113]:
def make_correlated_features(num_samples,p,rho):
  vcor = []
  for i in range(p):
    vcor.append(rho**i)
  r = toeplitz(vcor)
  mu = np.repeat(0,p)
  x = np.random.multivariate_normal(mu, r, size=num_samples)
  return x

In [114]:
rho =0.9
p = 10
n = 500
vcor = []
for i in range(p):
  vcor.append(rho**i)

In [115]:
x = make_correlated_features(n,p,rho)

In [116]:
beta =np.array([-1,2,3,0,0,0,0,2,-1,4])
beta = beta.reshape(-1,1)
betastar = np.concatenate([beta,np.repeat(0,p-len(beta)).reshape(-1,1)],axis=0)

In [117]:
# generate response with noise
y = x@betastar + 1.5*np.random.normal(size=(n,1))

# put your data in torch tensors
x = torch.tensor(x,device=device)
y = torch.tensor(y,device=device)

In [118]:
model = SCAD(input_size=x.shape[1],alpha=0.01)

In [119]:
model.fit(x,y,num_epochs=2000,learning_rate=0.01)

In [123]:
SCAD_Co = model.get_coefficients()[0].cpu().detach().numpy()

In [124]:
model = ElasticNet(input_size=x.shape[1],alpha=0.01)
model.fit(x,y,num_epochs=2000,learning_rate=0.01)
Elastic_Co = model.get_coefficients()[0].cpu().detach().numpy()

Epoch [100/2000], Loss: 2.6214340199516633
Epoch [200/2000], Loss: 2.1328362420518485
Epoch [300/2000], Loss: 1.8841879458946242
Epoch [400/2000], Loss: 1.738314444590891
Epoch [500/2000], Loss: 1.6452390431458916
Epoch [600/2000], Loss: 1.5804088646283945
Epoch [700/2000], Loss: 1.5324637572057693
Epoch [800/2000], Loss: 1.496258926744655
Epoch [900/2000], Loss: 1.4677268898122482
Epoch [1000/2000], Loss: 1.4447962897074702
Epoch [1100/2000], Loss: 1.4261321389380908
Epoch [1200/2000], Loss: 1.4107950373247689
Epoch [1300/2000], Loss: 1.398098877695664
Epoch [1400/2000], Loss: 1.3875275898644694
Epoch [1500/2000], Loss: 1.3786838865013757
Epoch [1600/2000], Loss: 1.3717332596859941
Epoch [1700/2000], Loss: 1.3663143377312097
Epoch [1800/2000], Loss: 1.3617481867069083
Epoch [1900/2000], Loss: 1.357887323940999
Epoch [2000/2000], Loss: 1.354613622909346


In [125]:
model = SqrtLasso(input_size=x.shape[1],alpha=0.01)
model.fit(x,y,num_epochs=2000,learning_rate=0.01)
Lasso_Co = model.get_coefficients()[0].cpu().detach().numpy()

Epoch [100/2000], Loss: 2.684551740627864
Epoch [200/2000], Loss: 2.3807414382571683
Epoch [300/2000], Loss: 2.145101612588841
Epoch [400/2000], Loss: 1.9815766870635745
Epoch [500/2000], Loss: 1.877291192546535
Epoch [600/2000], Loss: 1.807766792868032
Epoch [700/2000], Loss: 1.7577910323797437
Epoch [800/2000], Loss: 1.7218814705191994
Epoch [900/2000], Loss: 1.6966963898880794
Epoch [1000/2000], Loss: 1.6830358804565846
Epoch [1100/2000], Loss: 1.6767047638730699
Epoch [1200/2000], Loss: 1.6733238555529504
Epoch [1300/2000], Loss: 1.6714923689545564
Epoch [1400/2000], Loss: 1.6705449565325527
Epoch [1500/2000], Loss: 1.6700774220089645
Epoch [1600/2000], Loss: 1.6698575631763601
Epoch [1700/2000], Loss: 1.6697598264873654
Epoch [1800/2000], Loss: 1.6697179395379493
Epoch [1900/2000], Loss: 1.6697015766798884
Epoch [2000/2000], Loss: 1.6696952618367256


In [128]:
np.mean(SCAD_Co)

0.8940140037978537

In [129]:
np.mean(Elastic_Co)

0.8924425046208704

In [130]:
np.mean(Lasso_Co)

0.8940086592884766

With these randomly generated values, SCAD outperformed Lasso which outperformed Elastic Net.